- 필요한 패키지 임포트

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import FinanceDataReader as fdr
import io 
import zipfile
import xml.etree.ElementTree as et 
import json
import requests 

- api 넘버 가져오기

In [3]:
api_num = pd.read_csv('/Users/kimhansaem/workspace/11.DataAnalysis/01.데이터분석/data/apinumber.txt', sep='\s+', header=None,
                                names=['column_name'])
api_number = api_num['column_name']

- 상장 기업 전체 기업 이름, 기업코드 api 이용하여 불러오기 

In [4]:
def get_corpcode(crtfc_key): 
    params = {'crtfc_key':crtfc_key}
    url = "https://opendart.fss.or.kr/api/corpCode.xml" 
    items_en = ["corp_code", "corp_name", "stock_code", "modify_date"] 
    
    ## Column명을 한글로 하려면 아래부분 쌍따옴표안을 수정하면 됨 
    items_kr = ["corp_code", "corp_name", "stock_code", "modify_date"] 
    results = requests.get(url,params=params) 
    uzfile = zipfile.ZipFile(io.BytesIO(results.content))
    final = uzfile.open(uzfile.namelist()[0]) 
    root = et.fromstring(final.read().decode('utf-8')) 
    data = [] 
    for child in root: 
        if len(child.find('stock_code').text.strip()) > 1: 
            data.append([]) 
            for item in items_en: 
                data[-1].append(child.find(item).text) 
    df = pd.DataFrame(data, columns = items_kr) 
    ##print(df) 
    return df

- 다트에 있는 기업 리스트 추출

In [5]:
crtfc_key = api_number[0]
code_df = get_corpcode(crtfc_key)
code_df.head()

,corp_code,corp_name,stock_code,modify_date
0,00260985,한빛네트,036720,20170630
1,00264529,엔플렉스,040130,20170630
2,00358545,동서정보기술,055000,20170630
3,00231567,애드모바일,032600,20170630
4,00247939,씨모스,037600,20170630


- 다트에 재무제표 올라와 있는 기업 개수: 3418

In [6]:
code_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3425 entries, 0 to 3424
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   corp_code    3425 non-null   object
 1   corp_name    3425 non-null   object
 2   stock_code   3425 non-null   object
 3   modify_date  3425 non-null   object
dtypes: object(4)
memory usage: 107.2+ KB


- 반도체기업 이름, stock code 추출

In [7]:
# 반도체 기업 이름 추출
company_name_df = pd.read_csv('/Users/kimhansaem/workspace/11.DataAnalysis/01.데이터분석/data/반도체 기업 목록.txt', sep='\s+', header=None,
                                names=['column_index', 'column_name'])

In [8]:
# 반도체 기업 추출 확인
company_names = company_name_df['column_index'].values.tolist()
print(company_names[:5])

['큐에스아이', '예스티', '미코', '신성이엔지', '코세스']


- 반도체기업 주식코드 추출

In [9]:
df_corp = code_df[code_df['corp_name'].isin(company_names)]
df_corp.head(3)

,corp_code,corp_name,stock_code,modify_date
611,00686068,신성이엔지,104110,20171208
775,00625942,아이원스,114810,20220117
808,00484682,엘오티베큠,083310,20211123


- 신성이엔지 주식코드 수정

In [10]:
ss_mask = df_corp["corp_name"] =="신성이엔지"
df_corp.loc[ss_mask, "stock_code"] = "011930"
df_corp.head()

,corp_code,corp_name,stock_code,modify_date
611,00686068,신성이엔지,011930,20171208
775,00625942,아이원스,114810,20220117
808,00484682,엘오티베큠,083310,20211123
809,00492894,젬백스,082270,20211124
830,00479787,인텍플러스,064290,20211129


- 반도체기업 주식코드 수정 완료

In [11]:
df_corp2 = df_corp.drop(['modify_date'], axis=1)
df_corp2

,corp_code,corp_name,stock_code
611,00686068,신성이엔지,011930
775,00625942,아이원스,114810
808,00484682,엘오티베큠,083310
809,00492894,젬백스,082270
830,00479787,인텍플러스,064290
...,...,...,...
3331,00246417,이오테크닉스,039030
3343,00535676,테크윙,089030
3344,00659161,싸이맥스,160980
3413,00121686,엠케이전자,033160


In [12]:
df = pd.read_csv('./data/재무제표파일/aaa.csv')

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127 entries, 0 to 126
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   corp_code   127 non-null    int64 
 1   corp_name   127 non-null    object
 2   stock_code  127 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 3.1+ KB


- 반도체기업 stock_code 리스트로 가져오기

In [13]:
company_codes = df_corp2['stock_code'].values.tolist()
company_codes

['011930',
 '114810',
 '083310',
 '082270',
 '064290',
 '033170',
 '131290',
 '053610',
 '402340',
 '049080',
 '224060',
 '131970',
 '122640',
 '036540',
 '061970',
 '087600',
 '092070',
 '077500',
 '031980',
 '217190',
 '050090',
 '098120',
 '064520',
 '093640',
 '080520',
 '067310',
 '059120',
 '217500',
 '036930',
 '222160',
 '200710',
 '330860',
 '089790',
 '240810',
 '303030',
 '029460',
 '036200',
 '073570',
 '200470',
 '059090',
 '030530',
 '119830',
 '045300',
 '306620',
 '032580',
 '005290',
 '025560',
 '003160',
 '077360',
 '101160',
 '064760',
 '102120',
 '171010',
 '033640',
 '042700',
 '089530',
 '011930',
 '117670',
 '101400',
 '183300',
 '084370',
 '123010',
 '357780',
 '000990',
 '092600',
 '195870',
 '058470',
 '208860',
 '095340',
 '080220',
 '253590',
 '096610',
 '147760',
 '101490',
 '080580',
 '097800',
 '311320',
 '052860',
 '074600',
 '142210',
 '319660',
 '089890',
 '052900',
 '348210',
 '269620',
 '006200',
 '267320',
 '095610',
 '356860',
 '066310',
 '241790',

In [15]:
company_codes[2]

'083310'

- KRX 상장사 전체 불러오기

In [17]:
df = fdr.StockListing('KRX')
df.head()

,Symbol,Market,Name,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region
0,060310,KOSDAQ,3S,전자부품 제조업,반도체 웨이퍼 캐리어,2002-04-23,03월,김세완,http://www.3sref.com,서울특별시
1,095570,KOSPI,AJ네트웍스,산업용 기계 및 장비 임대업,"렌탈(파렛트, OA장비, 건설장비)",2015-08-21,12월,박대현,http://www.ajnet.co.kr,서울특별시
2,006840,KOSPI,AK홀딩스,기타 금융업,지주사업,1999-08-11,12월,"채형석, 이석주(각자 대표이사)",http://www.aekyunggroup.co.kr,서울특별시
3,054620,KOSDAQ,APS홀딩스,기타 금융업,인터넷 트래픽 솔루션,2001-12-04,12월,정기로,http://www.apsholdings.co.kr,경기도
4,265520,KOSDAQ,AP시스템,특수 목적용 기계 제조업,디스플레이 제조 장비,2017-04-07,12월,김영주,http://www.apsystems.co.kr,경기도


- 주가정보 수집

In [18]:
name = 'Open	High	Low	Close	Volume	Change'
tag = name.split()
tag

['Open', 'High', 'Low', 'Close', 'Volume', 'Change']

### 여기부터 자동으로 들어가게끔 for문 만들어야함

In [19]:
# 한 기업 넣는 거//  for문애서는 i+1로 변경
company_codes2 = df_corp2['stock_code'][:1].values.tolist()
company_codes2

['011930']

- 5년치 주가 조회

In [24]:
# 2017이 시작연도 2022가 끝연도라서 총 5~6년치 조회
item_list = []
for code in company_codes2:
    stockpriceinfo = fdr.DataReader(code, "2018", "2022")[tag]
    item_list.append(stockpriceinfo)

In [25]:
df_onecorp = pd.concat(item_list, axis=0)
df_onecorp.columns = tag
df_onecorp.tail()

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2021-12-24,2170,2170,2095,2105,1728317,-0.020930
2021-12-27,2105,2125,2075,2075,1253775,-0.014252
2021-12-28,2080,2155,2060,2090,2134322,0.007229
2021-12-29,2115,2130,2075,2095,1233435,0.002392
2021-12-30,2105,2160,2095,2145,1638535,0.023866


In [26]:
# 5년치 다 나오는지 확인
df_onecorp.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 986 entries, 2018-01-02 to 2021-12-30
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    986 non-null    int64  
 1   High    986 non-null    int64  
 2   Low     986 non-null    int64  
 3   Close   986 non-null    int64  
 4   Volume  986 non-null    int64  
 5   Change  986 non-null    float64
dtypes: float64(1), int64(5)
memory usage: 53.9 KB


In [ ]:
# 기업코드가 이름으로 저장되게끔 company_codes[i]+1
def get_total_price(item_code):
    for item_code in company_codes:
        every_price = fdr.DataReader(item_code, "2017", "2022")[tag]
 

In [55]:
!pip install sqlalchemy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 20.7 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for sqlalchemy: filename=SQLAlchemy-1.4.32-cp38-cp38-macosx_11_0_arm64.whl size=1536908 sha256=05412114bc399b2f8f831d9a1216eb03f0a40ff499740bd0a56341c7155bc7b3
  Stored in directory: /Users/kimhansaem/Library/Caches/pip/wheels/f1/89/a6/de94839bd1eae1e97da3e026f85938bc8afca6739bda1df206
Successfully built sqlalchemy
You should consider upgrading via the '/Users/kimhansaem/miniconda3/bin/python -m pip install --upgrade pip' command.


In [56]:
import MySQLdb
from sqlalchemy import create_engine

In [58]:
engine = create_engine("mysql+mysqldb://호스트이름(mhso같은거):"+"비번"+"@아이피주소/저장할db이름", encoding='utf-8')
conn = engine.connect()

In [ ]:
이 사이에 """create table """ 컬럼 설정하는 코드 넣으면됨 

In [62]:
# MySQL에 저장하기
# pandas의 to_sql() 함수 사용 저장
# DB하이텍 연습
df.to_sql(name='price_list', con=engine, if_exists='append')

In [ ]:
# Base : 기본적으로 테이블을 다루기 위한 여러 내용들이 들어있음.
# - 대표적으로 CRUD에 대한 메소드가 들어있다.
Base = declarative_base() # Base 클래스를 얻어내는 함수

# Base 클래스 상속
# 데이터만 담겨있는 클래스(객체) : Value Object VO
class User(Base):
    
    __tablename__ = "price_list" # 테이블 이름
    
    # 컬럼 정의
    #  - 컬럼명, 컬럼 데이터 타입
    Date = Column((DateTime) # primary_key=True
    Open = Column(Integer) 
    High = Column(Integer)
    Low = Column(Integer)
    Close = Column(Integer)
    Change = Column(Integer)
    
    # 생성자에서는 각 컬럼에 데이터를 매핑
    def __init__(self, name, email, age, rdate):
        # self.변수는 컬럼명과 반드시 일치
        self.name = name
        self.email = email
        self.age = age
        self.rdate = rdate
    
    # repr 함수
    def __repr__(self):
        return "<User {}, {}, {}, {}>".format(self.name, self.email, self.age, self.rdate)